<a href="https://colab.research.google.com/github/tinybeachthor/ucboulder-ml-supervised/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd
from datetime import datetime

## Load data

https://www.kaggle.com/datasets/sujaykapadnis/top-100-billboard

This dataset contains:
- Billboard 100 data from 1958 to 2021
- Spotify audio features for songs


In [2]:
!git clone https://github.com/tinybeachthor/ucboulder-ml-supervised
%cd ucboulder-ml-supervised

Cloning into 'ucboulder-ml-supervised'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10 (delta 0), reused 7 (delta 0), pack-reused 0
Receiving objects: 100% (10/10), 12.38 MiB | 17.46 MiB/s, done.
/content/ucboulder-ml-supervised


In [3]:
!unzip data/billboard.csv.zip
!unzip data/audio_features.csv.zip

Archive:  data/billboard.csv.zip
  inflating: billboard.csv           
Archive:  data/audio_features.csv.zip
  inflating: audio_features.csv      


In [5]:
billboard = pd.read_csv('billboard.csv')
billboard.head()

,url,week_id,week_position,song,performer,song_id,instance,previous_week_position,peak_position,weeks_on_chart
0,http://www.billboard.com/charts/hot-100/1965-0...,7/17/1965,34,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,45.0,34,4
1,http://www.billboard.com/charts/hot-100/1965-0...,7/24/1965,22,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,34.0,22,5
2,http://www.billboard.com/charts/hot-100/1965-0...,7/31/1965,14,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,22.0,14,6
3,http://www.billboard.com/charts/hot-100/1965-0...,8/7/1965,10,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,14.0,10,7
4,http://www.billboard.com/charts/hot-100/1965-0...,8/14/1965,8,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,10.0,8,8


In [6]:
audio_features = pd.read_csv('audio_features.csv')
audio_features.head()

,song_id,performer,song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_duration_ms,spotify_track_explicit,spotify_track_album,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
0,-twistin'-White Silver SandsBill Black's Combo,Bill Black's Combo,-twistin'-White Silver Sands,[],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,¿Dònde Està Santa Claus? (Where Is Santa Claus...,Augie Rios,¿Dònde Està Santa Claus? (Where Is Santa Claus?),['novelty'],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,......And Roses And RosesAndy Williams,Andy Williams,......And Roses And Roses,"['adult standards', 'brill building pop', 'eas...",3tvqPPpXyIgKrm4PR9HCf0,https://p.scdn.co/mp3-preview/cef4883cfd1e0e53...,166106.0,False,The Essential Andy Williams,0.154,...,-14.063,1.0,0.0315,0.91100,0.000267,0.112,0.150,83.969,4.0,38.0
3,...And Then There Were DrumsSandy Nelson,Sandy Nelson,...And Then There Were Drums,"['rock-and-roll', 'space age pop', 'surf music']",1fHHq3qHU8wpRKHzhojZ4a,NaN,172066.0,False,Compelling Percussion,0.588,...,-17.278,0.0,0.0361,0.00256,0.745000,0.145,0.801,121.962,4.0,11.0
4,...Baby One More TimeBritney Spears,Britney Spears,...Baby One More Time,"['dance pop', 'pop', 'post-teen pop']",3MjUtNVVq3C8Fn0MP3zhXa,https://p.scdn.co/mp3-preview/da2134a161f1cb34...,211066.0,False,...Baby One More Time (Digital Deluxe Version),0.759,...,-5.745,0.0,0.0307,0.20200,0.000131,0.443,0.907,92.960,4.0,77.0


## Clean data


### Billboard

- Convert `week_id` to datetime
- Filter data from 2010 to 2020
- Group into single row per song
- Construct columns describing songs (first week on chart, last week on chart, highest rank, lowest rank, ...)

In [10]:
billboard.dtypes

url                        object
week_id                    object
week_position               int64
song                       object
performer                  object
song_id                    object
instance                    int64
previous_week_position    float64
peak_position               int64
weeks_on_chart              int64
dtype: object

In [13]:
billboard['week'] = pd.to_datetime(billboard.week_id)

In [21]:
start_date = datetime(2010, 1, 1)
end_date = datetime(2020, 1, 1)
billboard_2010s = billboard[billboard['week'].between(start_date, end_date)]

In [40]:
billboard_2010s_min = billboard_2010s.groupby(by=['song_id']).min()
billboard_2010s_max = billboard_2010s.groupby(by=['song_id']).max()

In [48]:
billboard_clean = pd.DataFrame({
    'song': billboard_2010s_min.song,
    'performer': billboard_2010s_min.performer,

    'first_week_on_chart': billboard_2010s_min.week,
    'last_week_on_chart': billboard_2010s_max.week,
    'weeks_on_chart': billboard_2010s_max.weeks_on_chart,

    'highest_position': billboard_2010s_min.peak_position,
    'lowest_position': billboard_2010s_max.week_position,
}).sort_values(by=['first_week_on_chart'])

billboard_clean.head(10)

,song,performer,first_week_on_chart,last_week_on_chart,weeks_on_chart,highest_position,lowest_position
song_id,,,,,,,
Sexy ChickDavid Guetta Featuring Akon,Sexy Chick,David Guetta Featuring Akon,2010-01-02,2010-05-22,40,5,50
Russian RouletteRihanna,Russian Roulette,Rihanna,2010-01-02,2010-02-06,14,9,100
Give It Up To MeShakira Featuring Lil Wayne,Give It Up To Me,Shakira Featuring Lil Wayne,2010-01-02,2010-02-06,11,29,99
Say AahTrey Songz Featuring Fabolous,Say Aah,Trey Songz Featuring Fabolous,2010-01-02,2010-06-12,29,9,43
Heartbreak WarfareJohn Mayer,Heartbreak Warfare,John Mayer,2010-01-02,2010-05-08,20,34,93
Hell Breaks LooseEminem Featuring Dr. Dre,Hell Breaks Loose,Eminem Featuring Dr. Dre,2010-01-02,2010-01-02,1,29,29
Kings And QueensThirty Seconds To Mars,Kings And Queens,Thirty Seconds To Mars,2010-01-02,2010-01-09,5,82,100
Sex TherapyRobin Thicke,Sex Therapy,Robin Thicke,2010-01-02,2010-05-08,19,54,100
You Belong With MeTaylor Swift,You Belong With Me,Taylor Swift,2010-01-02,2010-04-17,50,2,50


### Audio features

- Drop rows without data (`spotify_track_id` missing)
- Select interesting columns

In [50]:
audio_features.index = audio_features['song_id']

In [53]:
audio_features.columns

Index(['song_id', 'performer', 'song', 'spotify_genre', 'spotify_track_id',
       'spotify_track_preview_url', 'spotify_track_duration_ms',
       'spotify_track_explicit', 'spotify_track_album', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'spotify_track_popularity'],
      dtype='object')

In [63]:
audio_features_clean = audio_features[audio_features.spotify_track_id.notna()]
audio_features_clean = audio_features_clean.drop(columns=[
    'song_id', 'performer', 'song',
    'spotify_genre',
    'spotify_track_id',
    'spotify_track_preview_url',
    'spotify_track_popularity',
    'spotify_track_album',
])

audio_features_clean.head()

,spotify_track_duration_ms,spotify_track_explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
song_id,,,,,,,,,,,,,,
......And Roses And RosesAndy Williams,166106.0,False,0.154,0.185,5.0,-14.063,1.0,0.0315,0.91100,0.000267,0.112,0.150,83.969,4.0
...And Then There Were DrumsSandy Nelson,172066.0,False,0.588,0.672,11.0,-17.278,0.0,0.0361,0.00256,0.745000,0.145,0.801,121.962,4.0
...Baby One More TimeBritney Spears,211066.0,False,0.759,0.699,0.0,-5.745,0.0,0.0307,0.20200,0.000131,0.443,0.907,92.960,4.0
...Ready For It?Taylor Swift,208186.0,False,0.613,0.764,2.0,-6.509,1.0,0.1360,0.05270,0.000000,0.197,0.417,160.015,4.0
'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,205560.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Merge

- Merge into single DataFrame

In [69]:
df = billboard_clean.merge(audio_features_clean, how='left', left_index=True, right_index=True)
df.head()

,song,performer,first_week_on_chart,last_week_on_chart,weeks_on_chart,highest_position,lowest_position,spotify_track_duration_ms,spotify_track_explicit,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
song_id,,,,,,,,,,,,,,,,,,,,,
#BeautifulMariah Carey Featuring Miguel,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,2013-09-07,16,15,95,199866.0,False,0.677,...,4.0,-5.405,1.0,0.0391,0.34600,0.000000,0.3470,0.469,107.042,4.0
#SELFIEThe Chainsmokers,#SELFIE,The Chainsmokers,2014-03-15,2014-05-24,11,16,95,183750.0,False,0.807,...,0.0,-3.282,1.0,0.2260,0.01340,0.000012,0.0787,0.651,127.973,4.0
#thatPOWERwill.i.am Featuring Justin Bieber,#thatPOWER,will.i.am Featuring Justin Bieber,2013-04-06,2013-07-20,16,17,85,279506.0,False,0.797,...,6.0,-6.096,0.0,0.0583,0.00112,0.000077,0.0748,0.403,128.000,4.0
$ave Dat MoneyLil Dicky Featuring Fetty Wap & Rich Homie Quan,$ave Dat Money,Lil Dicky Featuring Fetty Wap & Rich Homie Quan,2015-10-10,2016-03-19,19,71,100,290835.0,True,0.858,...,2.0,-5.361,1.0,0.2300,0.11400,0.000000,0.2240,0.364,98.012,4.0
'Til Summer Comes AroundKeith Urban,'Til Summer Comes Around,Keith Urban,2010-01-30,2010-05-15,16,58,94,331466.0,False,0.570,...,9.0,-7.608,0.0,0.0331,0.59300,0.000136,0.7700,0.308,127.907,4.0


In [68]:
df = df.sort_values(by=['first_week_on_chart'])
df.head(10)

,song,performer,first_week_on_chart,last_week_on_chart,weeks_on_chart,highest_position,lowest_position,spotify_track_duration_ms,spotify_track_explicit,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
song_id,,,,,,,,,,,,,,,,,,,,,
I Wanna Make You Close Your EyesDierks Bentley,I Wanna Make You Close Your Eyes,Dierks Bentley,2010-01-02,2010-02-27,20,52,87,239119.0,False,0.599,...,1.0,-5.745,1.0,0.0232,0.104000,0.000048,0.1150,0.359,95.979,4.0
I AmMary J. Blige,I Am,Mary J. Blige,2010-01-02,2010-05-08,20,55,94,237493.0,False,0.599,...,11.0,-4.663,0.0,0.0334,0.066300,0.000000,0.2070,0.576,82.251,4.0
You Belong With MeTaylor Swift,You Belong With Me,Taylor Swift,2010-01-02,2010-04-17,50,2,50,231133.0,False,0.687,...,6.0,-4.440,1.0,0.0386,0.162000,0.000013,0.1140,0.443,129.964,4.0
ReplayIyaz,Replay,Iyaz,2010-01-02,2010-04-24,34,2,46,182306.0,False,0.706,...,9.0,-6.323,1.0,0.0708,0.173000,0.000000,0.1680,0.195,91.031,4.0
Cryin' For Me (Wayman's Song)Toby Keith,Cryin' For Me (Wayman's Song),Toby Keith,2010-01-02,2010-03-13,12,73,91,285506.0,False,0.510,...,8.0,-7.685,0.0,0.0284,0.362000,0.001390,0.1120,0.261,156.080,4.0
Baby By Me50 Cent Featuring Ne-Yo,Baby By Me,50 Cent Featuring Ne-Yo,2010-01-02,2010-02-06,13,28,98,213360.0,True,0.637,...,0.0,-4.721,0.0,0.0773,0.004620,0.011600,0.0276,0.345,190.097,4.0
Red LightDavid Nail,Red Light,David Nail,2010-01-02,2010-01-23,16,54,83,246333.0,False,0.386,...,9.0,-2.468,1.0,0.0372,0.101000,0.000011,0.2110,0.428,163.907,3.0
I Wanna RockSnoop Dogg,I Wanna Rock,Snoop Dogg,2010-01-02,2010-04-10,15,41,100,235986.0,False,0.521,...,5.0,-3.238,1.0,0.3180,0.168000,0.000000,0.2890,0.431,122.852,5.0
Give It Up To MeShakira Featuring Lil Wayne,Give It Up To Me,Shakira Featuring Lil Wayne,2010-01-02,2010-02-06,11,29,99,183573.0,False,0.841,...,5.0,-6.311,0.0,0.0705,0.443000,0.000000,0.1220,0.688,115.984,4.0


## Export data

Export the cleaned and merged dataset.

In [71]:
filename = 'billboard_2010s.csv'

df.to_csv(filename)

Download the data.

In [72]:
from google.colab import files

files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>